<a href="https://colab.research.google.com/github/qwertymasterkey/captcha_recognition/blob/main/3_Captcha_model_resnet_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
from pathlib import Path
data_dir = Path("/content/drive/MyDrive/captcha_segment/segmented_dataset_")
images = sorted(list(map(str, list(data_dir.glob("*.jpg")))))

In [ ]:
!pip install split_folders
import splitfolders


In [ ]:
input_folder="/content/drive/MyDrive/captcha_segment/segmented_dataset_/captcha/data/"
output_folder="/content/drive/MyDrive/captcha_segment/splitted_dataset_/"


In [ ]:
splitfolders.ratio(input_folder,output_folder,seed=5,ratio=(.7,.15,.15))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
img_height,img_width=(224,224)
batch_size=32
train_data_dir="/content/drive/MyDrive/captcha_segment/splitted_dataset_/train"
val_data_dir="/content/drive/MyDrive/captcha_segment/splitted_dataset_/val"
test_data_dir="/content/drive/MyDrive/captcha_segment/splitted_dataset_/test"

In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 rotation_range=20,
                                 horizontal_flip=True,)

In [ ]:
train_generator=train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 4072 images belonging to 62 classes.


In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
valid_generator=test_datagen.flow_from_directory(
    val_data_dir,
    target_size=(img_height,img_width),
    batch_size=batch_size,
    class_mode='categorical',
)

Found 849 images belonging to 62 classes.


In [ ]:
test_generator=test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height,img_width),
    batch_size=1,
    class_mode='categorical',
)

Found 937 images belonging to 62 classes.


In [ ]:
x,y=test_generator.next()

In [ ]:
x.shape

(1, 224, 224, 3)

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization,GlobalAveragePooling2D
import numpy as np
from tensorflow.keras import layers


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation,BatchNormalization
import os
import pickle

In [ ]:
train_generator.num_classes

62

In [ ]:
base_model=ResNet50(include_top=False,weights='imagenet')
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x)
x=Dropout(0.2)(x)
predictions=Dense(train_generator.num_classes,activation='softmax')(x)
model=Model(inputs=base_model.input,outputs=predictions)

for layer in base_model.layers:
  layer.trainable=True
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

94781440/94765736 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                             

In [ ]:
filepath = 'my_best_model.epoch{epoch:02d}-loss{val_loss:.2f}.hdf5'
from keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_loss',
                             verbose=1,
                             save_best_only=True,
                             mode='min')

In [ ]:
callbacks = [checkpoint]
model.fit(train_generator,
          epochs = 100,
          validation_data = valid_generator,
          callbacks=callbacks
          )

Epoch 1/100
128/128 [==============================] - ETA: 0s - loss: 2.0053 - accuracy: 0.4833
Epoch 1: val_loss improved from inf to 10.09943, saving model to my_best_model.epoch01-loss10.10.hdf5
128/128 [==============================] - 851s 7s/step - loss: 2.0053 - accuracy: 0.4833 - val_loss: 10.0994 - val_accuracy: 0.1378
Epoch 2/100
128/128 [==============================] - ETA: 0s - loss: 0.6584 - accuracy: 0.7974
Epoch 2: val_loss improved from 10.09943 to 2.88274, saving model to my_best_model.epoch02-loss2.88.hdf5
128/128 [==============================] - 68s 533ms/step - loss: 0.6584 - accuracy: 0.7974 - val_loss: 2.8827 - val_accuracy: 0.4028
Epoch 3/100
128/128 [==============================] - ETA: 0s - loss: 0.4118 - accuracy: 0.8743
Epoch 3: val_loss improved from 2.88274 to 0.44573, saving model to my_best_model.epoch03-loss0.45.hdf5
128/128 [==============================] - 68s 531ms/step - loss: 0.4118 - accuracy: 0.8743 - val_loss: 0.4457 - val_accuracy: 0.83

In [ ]:
test_loss,test_acc=model.evaluate(test_generator,verbose=1)

937/937 [==============================] - 176s 187ms/step - loss: 0.1454 - accuracy: 0.9851
